In [3]:
import pandas as pd

# Load the uploaded dataset
data = pd.read_csv('/content/HAM10000_metadata.csv')

# Display the first few rows of the dataset to understand its structure
print(data.head())  # Use print() to display rows in standard output


     lesion_id      image_id   dx dx_type   age   sex localization
0  HAM_0000118  ISIC_0027419  bkl   histo  80.0  male        scalp
1  HAM_0000118  ISIC_0025030  bkl   histo  80.0  male        scalp
2  HAM_0002730  ISIC_0026769  bkl   histo  80.0  male        scalp
3  HAM_0002730  ISIC_0025661  bkl   histo  80.0  male        scalp
4  HAM_0001466  ISIC_0031633  bkl   histo  75.0  male          ear


In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Load the dataset
data = pd.read_csv('/content/HAM10000_metadata.csv')

# Preprocessing
# Exclude 'lesion_id' and 'image_id' columns from features
X = data[['age', 'sex', 'localization']].values  # Numerical and categorical features
y = data.iloc[:, -1].values   # Target (dx)

# Convert categorical features to numerical using OneHotEncoder
encoder_features = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X = encoder_features.fit_transform(X)

# Encode target labels to one-hot format
encoder_target = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
y = encoder_target.fit_transform(y.reshape(-1, 1))

# Standardize the numerical features (now represented within the OneHotEncoded X)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the Neural Network Architecture
input_size = X.shape[1]  # Number of input features
hidden_size = 5          # Number of neurons in the hidden layer
output_size = y.shape[1] # Number of output classes (dx)

# Initialize weights and biases
np.random.seed(42)
weights_input_hidden = np.random.randn(input_size, hidden_size)
bias_hidden = np.zeros((1, hidden_size))
weights_hidden_output = np.random.randn(hidden_size, output_size)
bias_output = np.zeros((1, output_size))

# Activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def softmax(x):
    exp_values = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_values / np.sum(exp_values, axis=1, keepdims=True)

# Loss function
def cross_entropy_loss(y_true, y_pred):
    return -np.mean(np.sum(y_true * np.log(y_pred + 1e-9), axis=1))

# Forward pass
def forward_pass(X):
    hidden_input = np.dot(X, weights_input_hidden) + bias_hidden
    hidden_output = sigmoid(hidden_input)

    output_input = np.dot(hidden_output, weights_hidden_output) + bias_output
    output_output = softmax(output_input)

    return hidden_output, output_output

# Backpropagation
def backward_pass(X, y, hidden_output, output_output, learning_rate):
    global weights_input_hidden, bias_hidden, weights_hidden_output, bias_output

    # Output layer error
    output_error = output_output - y
    output_gradient = np.dot(hidden_output.T, output_error)

    # Hidden layer error
    hidden_error = np.dot(output_error, weights_hidden_output.T) * sigmoid_derivative(hidden_output)
    hidden_gradient = np.dot(X.T, hidden_error)

    # Update weights and biases
    weights_hidden_output -= learning_rate * output_gradient
    bias_output -= learning_rate * np.sum(output_error, axis=0, keepdims=True)
    weights_input_hidden -= learning_rate * hidden_gradient
    bias_hidden -= learning_rate * np.sum(hidden_error, axis=0, keepdims=True)

# Training the Neural Network
epochs = 1000
learning_rate = 0.001

for epoch in range(epochs):
    # Forward pass
    hidden_output, output_output = forward_pass(X_train)

    # Compute loss
    loss = cross_entropy_loss(y_train, output_output)

    # Backward pass
    backward_pass(X_train, y_train, hidden_output, output_output, learning_rate)

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# Testing the Model
_, y_pred = forward_pass(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

accuracy = np.mean(y_pred_classes == y_true_classes)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 0, Loss: 3.3157
Epoch 100, Loss: 0.0948
Epoch 200, Loss: 0.0500
Epoch 300, Loss: 0.0360
Epoch 400, Loss: 0.0296
Epoch 500, Loss: 0.0256
Epoch 600, Loss: 0.0232
Epoch 700, Loss: 0.0212
Epoch 800, Loss: 0.0194
Epoch 900, Loss: 0.0189
Test Accuracy: 99.80%
